In [4]:
import os

from regex import regex
from collections import Counter

In [5]:
data_dir = 'data'

In [6]:
def read_file(path):
    with open(path, 'r') as f:
        lines = f.readlines()
        content = ''.join(lines)
        return content

In [25]:
def resolve_title(bill_content):
    """
    Extract the title from the beginning of a document.
    """
    
    try:
        title_pattern = r'U\s*S\s*T\s*A\s*W\s*A[\s\d\p{L}\p{P}]+?(?=Rozdział|Art)'
        title = regex.search(title_pattern, bill_content).group(0)
        cleansed = regex.sub('\s+', ' ', title)
        cleansed = regex.sub('U\s*S\s*T\s*A\s*W\s*A', 'Ustawa', cleansed).strip(' ')
        return cleansed
    except:
        return None

In [8]:
class BillFile:
    def __init__(self, file_name, content):
        self.file_name = file_name
        self.content = content
        self.title = resolve_title(content)

def read_files():
    files = {}
    for file_name in os.listdir(data_dir):
        path = os.path.join(data_dir, file_name)
        content = read_file(path)
        files[file_name] = BillFile(file_name, content)
    return files

In [160]:
class BillReference:
    def __init__(self, position, number, year, title=None):
        self.position = position
        self.number = number
        self.year = year
        self.title = title
        
    def pretty(self):
        title = '' if self.title is None else ', title: {0}...'.format(self.title[:10])
        return 'pos: {0}, nr: {1}, year: {2}'.format(self.position, self.number, self.year, title)
    
    def __repr__(self):
        return '{0}-{1}-{2}'.format(self.position, self.number, self.year)
    
    def __eq__(self, other):
        return self.position == other.position and self.year == other.year
    
    def __hash__(self):
        return self.__repr__().__hash__()

## 1. External references
---

In [168]:
def positions(text):
    nr_parts_pattern = r'Nr\s*(\d*)\,(\s*[^N]*)'
    nr_parts_matches = list(regex.finditer(nr_parts_pattern, text))
    
    results = []
    
    for nr_match in nr_parts_matches:
        nr = nr_match.group(1)
        rest = nr_match.group(2)
        
        position_pattern = r'\d+'
        positions = regex.findall(position_pattern, rest)
        
        for pos in positions:
            results.append((pos, nr))
            
    return results

In [169]:
def references_in_year_groups(text):
    references = []
    
    year_group_pattern = r'z\s(\d{4})\sr.\s((Nr\s*(\d*),\s*poz\.\s*(\d*)[,\si\d]*)*)'
    year_group_matches = list(regex.finditer(year_group_pattern, text))

    for year_group_match in year_group_matches:
        year = year_group_match.group(1)
        rest = year_group_match.group(2)
        
        for pos, nr in positions(rest):
            references.append(BillReference(pos, nr, year))
    return references

In [170]:
def handle_b(year, title, text):
    """
    Given something like:
      year = 2017
      title = o zmianie czegośtam
      text = Dz. U. Nr 5, pos 13 i 14
      
    Return:
      [BillReference(13, 5, 2017, 'o zmianie ustawy'),
       BillReference(13, 5, 2017, 'o zmianie ustawy')]
    """
    references = []
    for pos, nr in positions(text):
        references.append(BillReference(pos, nr, year, title))
        
    return references

In [171]:
def external_references(year, title, rest_match):
    """
    rest_match   - something like "Dz. U. z 2004 r. (...)"
    year & title - year and title that were match befor the rest_match 
                   like in:  (...) 29 lipca 2017 r. o zmianie ustawy (Dz. U. (...))
                   in which case year = 2017, title = "o zmianie ustawy", rest_match = "Dz. U. (...)"
    
    there are two cases:
    a) full reference
       when rest_match start's with a year: Dz. U. z 2016 r. poz. 1510 i 2074
       in which case we take "2016" as a year
    b) partial reference
       when rest_match start's with a Nr:  Dz. U. Nr 183, poz. 1538 
       in which case the year was somewhere before
    """

    references = []
    
    is_b = len(list(regex.finditer(r'Dz\.\s*U\.\s*z\s*\d{4}', rest_match))) == 0
    
    if is_b:
        year_matches = list(regex.finditer(r'z\s*\d{4}\s*r\.', rest_match))
        if len(year_matches) == 0:
            references += handle_b(year, title, text)
        else:
            start = year_matches[0].start()
            references += handle_b(year, title, text[:start])
            references += references_in_year_groups(rest_match[start:])
    else:
        references += references_in_year_groups(rest_match)
    
    return references

In [172]:
def external_journal_matches(text):
    """    
    Given a text like:
    
    Art. 2.
    W ustawie z dnia
    29 lipca 2005 r. o obrocie instrumentami finansowymi (Dz. U. Nr 183, poz. 1538,
    z późn. zm.[3]))
    w art. 70 w ust. 2 pkt 1 otrzymuje brzmienie:
    „1) art. 69
    
    It matches:
    - "2005" as a year
    - "o obrocie instrumentami finansowymi" as a title
    - (Dz. U (...)) - as something that will be processed later
    
    """
    
    references = []
    
    pattern = r'(?<=(\d{4})\sr\.)([\p{L}\p{P}\s]*)(\(Dz.\s?U.[^\)]*\))'
    matches = list(regex.finditer(pattern, text))
     
    for match in matches:
        year = match.group(1)
        title = match.group(2)
        rest = match.group(3).strip('()')
        
        references += external_references(year, title, rest)
        
    return references 

In [180]:
test = files['1994_215.txt']

for ref in external_journal_matches(test.content):
    print(ref.pretty())

pos: 82, nr: 11, year: 1991
pos: 113, nr: 26, year: 1992
pos: 254, nr: 54, year: 1992
pos: 585, nr: 127, year: 1993
pos: 3, nr: 1, year: 1994


In [173]:
def external_footnote_matches(text):
    """
     Matches the part: Dz. U. z 2004 r. (...)
     
     In foot notes like:
     
    [4]) Zmiany tekstu jednolitego wymienionej ustawy zostały
    ogłoszone w Dz. U. z 2004 r. Nr 273, poz. 2703, z 2005 r. Nr 155,
    poz. 1297 i Nr 172, poz. 1440, z 2006 r. Nr 12, poz. 61, z 2007 r. Nr 23, poz.
    136 i Nr 99, poz. 666, z 2008 r. Nr 218, poz. 1391 oraz z 2009 r. Nr 3, poz.
    11, Nr 19, poz. 101, Nr 86, poz. 720, Nr 105, poz. 877, Nr 115, poz. 966,
    Nr 143, poz. 1164 i Nr 157, poz. 1241.
    """
    
    references = []
    
    footnote_pattern = r'\[\d*\]([\s\p{L}\p{P}]*(?=Dz\.\s*U\.))(Dz\.\s*U\.\s*(z\s*\d{4}\s*r\.\s*|Nr\s*\d*[,\.]\s*|poz\.\s*\d*[,\.]?\s*|i\s*\d*,?\s*|oraz\s*)*)'
    footnote_matches = regex.finditer(footnote_pattern, text)

    for match in footnote_matches:
        references += references_in_year_groups(match.group(2))
    
    return references

In [174]:
def all_external_references(bill_content): 
    return external_journal_matches(bill_content) + external_footnote_matches(bill_content)

In [175]:
def print_external_references_in(bill):
    """
    For a given bill summarize the references to other bills
    """
    references = all_external_references(bill.content)
    reference_count = sorted(list(Counter(references).items()), key=lambda x: x[1], reverse=True)
    
    print('{0} - {1}...'.format(bill.file_name, bill.title[:30]))
    for ref, count in reference_count:
        print('\t{0:15}: {1}'.format(str(ref), count))

In [177]:
print_external_references_in(files['2004_962.txt'])

2004_962.txt - Ustawa z dnia 20 listopada 200...
	2571-256-2004  : 2
	2703-273-2004  : 2
	1297-155-2005  : 2
	1440-172-2005  : 2
	61-12-2006     : 2
	136-23-2007    : 2
	666-99-2007    : 2
	1391-218-2008  : 2
	11-3-2009      : 2
	101-19-2009    : 2
	720-86-2009    : 2
	877-105-2009   : 2
	966-115-2009   : 2
	1164-143-2009  : 2
	1241-157-2009  : 2
	82-11-2005     : 1
	32-11-2005     : 1
	210-30-2005    : 1
	82-11-2003     : 1
	32-11-2003     : 1
	210-30-2003    : 1
	708-104-2006   : 1
	1119-157-2006  : 1
	1056-171-2008  : 1
	69-13-2009     : 1
	341-42-2009    : 1
	649-77-2009    : 1
	659-78-2009    : 1
	1316-165-2009  : 1
	1317-166-2009  : 1
	1323-168-2009  : 1


In [178]:
def summary(bills):
    references = []
    for bill in bills:
        references += all_external_references(bill.content)
    reference_count = sorted(list(Counter(references).items()), key=lambda x: x[1], reverse=True)
    for ref, count in reference_count:
        print('\t{0:15}: {1}'.format(ref.pretty(), count))

In [151]:
summary(files.values())

	668-106-1998   : 673
	496-106-1996   : 450
	770-121-1997   : 409
	136-12-2000    : 348
	554-88-1997    : 270
	153-28-1997    : 235
	198-34-1990    : 235
	1268-120-2000  : 202
	943-141-1997   : 174
	1118-162-1998  : 160
	254-54-1992    : 153
	708-104-2006   : 151
	1126-162-1998  : 145
	550-48-2000    : 139
	110-24-1996    : 138
	756-117-1998   : 133
	769-121-1997   : 133
	17-4-1995      : 132
	192-35-1989    : 130
	676-74-2002    : 128
	272-43-1997    : 128
	509-105-1994   : 127
	711-104-2006   : 117
	926-137-1997   : 115
	163-34-1995    : 113
	1255-110-1999  : 110
	591-121-1994   : 106
	931-83-1999    : 105
	778-70-1999    : 103
	163-43-1994    : 101
	189-43-1996    : 101
	3-1-1994       : 99
	96-27-1994     : 93
	471-75-1997    : 91
	802-72-1999    : 89
	1041-95-2000   : 89
	1271-153-2002  : 88
	679-106-1997   : 87
	646-134-1993   : 86
	1178-101-1999  : 84
	959-96-2004    : 83
	636-60-1999    : 82
	360-38-1999    : 81
	64-8-2001      : 81
	1037-94-2000   : 79
	324-41-1988    : 79
	94

	176-32-1998    : 2
	498-45-2001    : 2
	693-99-2006    : 2
	471-95-1995    : 2
	1011-141-2006  : 2
	1059-178-2011  : 2
	1202-205-2011  : 2
	1504-153-2003  : 2
	1686-200-2002  : 2
	872-80-2001    : 2
	696-100-2006   : 2
	1368-191-2007  : 2
	1445-200-2007  : 2
	1180-192-2008  : 2
	1008-141-2006  : 2
	1829-249-2006  : 2
	26-11-2000     : 2
	503-80-1997    : 2
	166-38-1993    : 2
	208-34-1997    : 2
	1042-158-1997  : 2
	716-113-1998   : 2
	724-69-2001    : 2
	89-14-1990     : 2
	394-78-1992    : 2
	201-46-1991    : 2
	1231-147-2002  : 2
	719-80-2003    : 2
	802-75-2001    : 2
	1153-124-2003  : 2
	299-32-2002    : 1
	1500-180-2002  : 1
	973-89-2001    : 1
	389-44-2003    : 1
	979-112-2002   : 1
	1644-147-2001  : 1
	466-49-2004    : 1
	574-62-2004    : 1
	247-56-1983    : 1
	258-33-1937    : 1
	139-15-1936    : 1
	259-33-1937    : 1
	48-14-1980     : 1
	108-18-1987    : 1
	479-99-1994    : 1
	184-32-1976    : 1
	79-14-1986     : 1
	1087-94-1999   : 1
	994-21-1998    : 1
	404-59-2007    : 1


## 2. Internal references
---

Find all internal references to regulations, e.g. art. 5 ust. 2, art. 5 ust. 7, etc. The result should exclude the internal numbering of the bill (e.g. Art. 1. W ustawie ...). The result should be aggregated by regulation ID (as described below) and sorted by descending number of reference counts inside particular bill. The bills should be sorted by descending number of internal references. The reference format should include all elements necessary to identify the regulation, e.g.:
- art. 1, ust. 2 - if an article inside the regulation is referenced,
- ust. 2 - if a paragraph inside the same article is referenced,
- etc.

---

In [105]:
class ArticleReference:
    def __init__(self, paragraph, article, same_article):
        self.article = article
        self.paragraph = paragraph
        self.same_article = same_article

### Different corner cases

From 2004_964.txt:

ust. 1-3  
w art. 343 ust. 2, 3 i 5  
art. 185 ust. 1a 

In [36]:
test = files['2004_962.txt']

In [45]:
def search_for_internal(bill):
    articles_pattern = r'Art\.\s*\d*\.\s*[\p{L}\p{P}\d\s]*?(?=Art|\Z|\n+)'
    articles_matches = list(regex.finditer(articles_pattern, bill.content))
    
    for article in articles_matches:
        article_pattern =  r'art\.\s*(\d*)\s*ust\.\s*(\d*\w?(\s*(i|,|oraz)*\s*\d*)*)'
    
    return articles_matches

In [43]:
len(search_for_internal(test))

3

In [46]:
# (?<=art\.)\s*(\d*)[\s\p{L}\p{P}]*(ust\.\s*(\d)*)

## 3. Count occurrences of "ustawa"
---

In [21]:
def match_in_bills(bills):
    results = {}
    
    flexes = ['ustawa', 'ustawy', 'ustawy', 'ustaw', 'ustawie', 
              'ustawom',  'ustawę', 'ustawy', 'ustawą', 'ustawami',
              'ustawie', 'ustawach', 'ustawo', 'ustawy'] 
    
    alternatives = '|'.join(flexes)

    pattern = r'(?i)\b' + alternatives + '\b'
    
    for file_name, bill in bills.items():
        result = regex.findall(pattern, bill.content)
        results[file_name] = len(result)
        
    return results

In [22]:
def summary(results):
    least_n, least_v = None, None
    most_n, most_v = None, None
    total = 0
    for file_name, result in results.items():
        if least_v is None or least_v > result: 
            least_n, least_v = file_name, result
        if most_v is None or most_v < result:
            most_n, most_v = file_name, result
        total += result
        
    print('---------- SUMMARY ----------')
    print('least: {0:15} {1}'.format(least_n, least_v))
    print('most:  {0:15} {1}'.format(most_n, most_v))
    print('total: {0:21}'.format(total))
    print('-----------------------------')

In [23]:
results = match_in_bills(files)
summary(results)

---------- SUMMARY ----------
least: 1996_400.txt    0
most:  2000_696.txt    304
total:                 25940
-----------------------------
